In [1]:
from image_utils import get_pairs,get_dataset_info,estimate_dataset
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread

import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import backend as K


In [4]:
image_shape=(128,128,3)
main_folder_name='dataset'
max_positive_pairs_count=8000
max_negative_pairs_count=10000

In [3]:
get_dataset_info(main_folder_name)

-------------------------------------------------------
Dataset info

image shape =  (128, 128, 3)
image weight =  48.0  KiB
number of classes =  3
number of pictures =  132
positive pairs count = 7482
positive pairs images weight = 0.685  GiB
negative pairs count = 9942
negative pairs images weight = 0.91  GiB

total memory size = 1.5950000000000002  GiB


In [5]:
estimate_dataset(main_folder_name,
                 max_positive_pairs_count,
                 max_negative_pairs_count)

-------------------------------------------------------
Estimated info

desired positive pairs count = 8000
positive pairs images weight = 0.732  GiB

desired negative pairs count = 10000
negative pairs images weight = 0.916  GiB

total memory size = 1.6480000000000001  GiB


In [6]:
pairs,labels=get_pairs(main_folder_name,
                       max_positive_pairs_count,
                       max_negative_pairs_count)

Calculation began

Calculation is done

passed seconds:  9.806  seconds


In [7]:
def initialize_base_network():
    input = Input(shape=image_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1,)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)

    return Model(inputs=input, outputs=x)


def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [ ]:
def get_VGG():
    
    #model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=image_shape)
    model = Sequential(VGG16(weights='imagenet', include_top=False, input_shape=image_shape).layers)
    model.add(Flatten())
    model.add(Dense(1024,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(512,activation='relu'))
    model.add(Dropout(0.2))

    return model

In [ ]:
#base_network = initialize_base_network()
base_network = get_VGG()
base_network.summary()

In [ ]:

input_a = Input(shape=image_shape)
vect_output_a = base_network(input_a)

input_b = Input(shape=image_shape)
vect_output_b = base_network(input_b)


output = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([vect_output_a, vect_output_b])

# specify the inputs and output of the model
model = Model([input_a, input_b], output)

In [ ]:
model.summary()

In [ ]:
def contrastive_loss_with_margin(margin):
    def contrastive_loss(y_true, y_pred):
        square_pred = K.square(y_pred)
        margin_square = K.square(K.maximum(margin - y_pred, 0))
        return K.mean(y_true * square_pred + (1 - y_true) * margin_square)
    return contrastive_loss

In [ ]:
rms = RMSprop()
model.compile(loss=contrastive_loss_with_margin(margin=1), optimizer=rms)
history = model.fit([pairs[:,0],pairs[:,1]], 
                    labels, 
                    epochs=20, 
                    batch_size=16,
                    validation_split=0.05)